# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle
from keras.utils import to_categorical

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/3. normalized/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

x_train: (966, 26)
y_train: (966, 2)
x_test: (242, 26)
y_test: (242, 2)


In [3]:
pd.DataFrame(x_train).head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.226135,0.165461,0.218373,0.136944,0.218364,0.326976,0.467346,0.250757,0.120083,0.321016,...,0.100836,0.338785,0.407917,0.155705,0.151108,0.181358,0.077172,0.484244,0.416667,0.675275
1,0.079400,0.089118,0.061998,0.030564,0.062079,0.089922,0.191393,0.043280,0.024083,0.080073,...,0.071548,0.018426,0.228516,0.074497,0.071189,0.362160,0.183293,0.296468,0.083333,0.077304
2,0.103458,0.103848,0.100653,0.065950,0.100690,0.158392,0.283503,0.082511,0.046162,0.196872,...,0.033205,0.257567,0.234512,0.075839,0.075218,0.310095,0.049176,0.000000,0.000000,0.000000
3,0.146945,0.220966,0.091491,0.064243,0.091571,0.334515,0.488109,0.191163,0.105050,0.330666,...,0.032132,0.115608,0.119201,0.020805,0.020148,0.553460,0.109024,0.291681,0.000000,0.000000
4,0.130953,0.136021,0.060241,0.029970,0.060280,0.179025,0.251038,0.107355,0.057036,0.181617,...,0.076168,0.191104,0.285241,0.037584,0.035594,0.328655,0.115067,0.408358,0.083333,0.482370


In [4]:
pd.DataFrame(x_test).head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.092726,0.130350,0.048820,0.025074,0.048818,0.206552,0.278973,0.123197,0.069596,0.157321,...,0.030981,0.126401,0.125133,0.033557,0.032908,0.496924,0.083581,0.357097,0.000000,0.000000
1,0.017255,0.026758,0.015688,0.010905,0.015687,0.108347,0.153643,0.081450,0.033064,0.096120,...,0.003376,0.186343,0.123078,0.209396,0.208865,0.466729,0.007642,0.000000,0.000000,0.000000
2,0.178509,0.286687,0.142696,0.088427,0.142732,0.279412,0.436769,0.188018,0.079033,0.236513,...,0.027032,0.099181,0.093156,0.045638,0.042982,0.606600,0.101034,0.203158,0.166667,0.266331
3,0.156274,0.177721,0.077686,0.058234,0.077682,0.160971,0.343903,0.068792,0.033838,0.099012,...,0.083206,0.102144,0.217839,0.032215,0.030893,0.377669,0.122733,0.614431,0.000000,0.000000
4,0.240163,0.460682,0.150351,0.120549,0.150387,0.305724,0.386183,0.190613,0.076670,0.253248,...,0.025370,0.029456,0.043714,0.011409,0.010745,0.765075,0.141341,0.567164,0.000000,0.000000


# Tunable parameters

In [5]:
epochs = 5000
batch_size = 20
learning_rate = 0.0001
decay = 0.0001

# Autoencoder #1

In [6]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/results-autoencoder-01-02/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
def create_model(layers, dropout):
    global date
    x = Input(shape=(x_train.shape[1],))

    y = Dense(units = layers['layer_1'], activation='relu')(x)
    y = Dropout(dropout)(y)

    if (layers['layer_2'] > 0):
        y = Dense(units = layers['layer_2'], activation='relu')(y)
        y = Dropout(dropout)(y)
    
    y = Dense(units = x_train.shape[1], activation='sigmoid')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Layer 1': layers['layer_1'],
        'Layer 2': layers['layer_2'],
        'Dropout': dropout,
        'Learning Rate': learning_rate,
        'Decay': decay,
        'Epochs': epochs,
        'Batch size': batch_size
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [7]:
def get_callbacks():
    date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
    csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
    model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
    callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
    return callbacks

# Grid search

In [8]:
model = create_model({'layer_1': 10, 'layer_2': 0}, 0.4)
model.fit(x=x_train, y=x_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, x_test), callbacks=get_callbacks())

Train on 966 samples, validate on 242 samples
Epoch 1/5000
966/966 [==============================] - 0s - loss: 18.1016 - acc: 0.0383 - val_loss: 17.6237 - val_acc: 0.0165
Epoch 2/5000
966/966 [==============================] - 0s - loss: 18.0833 - acc: 0.0476 - val_loss: 17.6063 - val_acc: 0.0207
Epoch 3/5000
966/966 [==============================] - 0s - loss: 18.0608 - acc: 0.0487 - val_loss: 17.5894 - val_acc: 0.0248
Epoch 4/5000
966/966 [==============================] - 0s - loss: 18.0478 - acc: 0.0611 - val_loss: 17.5729 - val_acc: 0.0289
Epoch 5/5000
966/966 [==============================] - 0s - loss: 18.0269 - acc: 0.0745 - val_loss: 17.5560 - val_acc: 0.0413
Epoch 6/5000
966/966 [==============================] - 0s - loss: 18.0120 - acc: 0.0725 - val_loss: 17.5378 - val_acc: 0.0413
Epoch 7/5000
966/966 [==============================] - 0s - loss: 17.9939 - acc: 0.0839 - val_loss: 17.5187 - val_acc: 0.0372
Epoch 8/5000
966/966 [==============================] - 0s - loss

966/966 [==============================] - 0s - loss: 17.2645 - acc: 0.4389 - val_loss: 16.6395 - val_acc: 0.6653
Epoch 65/5000
966/966 [==============================] - 0s - loss: 17.2499 - acc: 0.4648 - val_loss: 16.6321 - val_acc: 0.6653
Epoch 66/5000
966/966 [==============================] - 0s - loss: 17.2475 - acc: 0.4586 - val_loss: 16.6253 - val_acc: 0.6653
Epoch 67/5000
966/966 [==============================] - 0s - loss: 17.2533 - acc: 0.4431 - val_loss: 16.6189 - val_acc: 0.6653
Epoch 68/5000
966/966 [==============================] - 0s - loss: 17.2399 - acc: 0.4555 - val_loss: 16.6119 - val_acc: 0.6653
Epoch 69/5000
966/966 [==============================] - 0s - loss: 17.2205 - acc: 0.4462 - val_loss: 16.6048 - val_acc: 0.6653
Epoch 70/5000
966/966 [==============================] - 0s - loss: 17.2106 - acc: 0.4317 - val_loss: 16.5979 - val_acc: 0.6653
Epoch 71/5000
966/966 [==============================] - 0s - loss: 17.2103 - acc: 0.4576 - val_loss: 16.5917 - val_ac

966/966 [==============================] - 0s - loss: 17.0095 - acc: 0.5559 - val_loss: 16.3363 - val_acc: 0.6653
Epoch 128/5000
966/966 [==============================] - 0s - loss: 16.9926 - acc: 0.5518 - val_loss: 16.3325 - val_acc: 0.6653
Epoch 129/5000
966/966 [==============================] - 0s - loss: 16.9908 - acc: 0.5600 - val_loss: 16.3290 - val_acc: 0.6653
Epoch 130/5000
966/966 [==============================] - 0s - loss: 17.0009 - acc: 0.5797 - val_loss: 16.3254 - val_acc: 0.6653
Epoch 131/5000
966/966 [==============================] - 0s - loss: 16.9951 - acc: 0.5600 - val_loss: 16.3222 - val_acc: 0.6653
Epoch 132/5000
966/966 [==============================] - 0s - loss: 16.9806 - acc: 0.5735 - val_loss: 16.3183 - val_acc: 0.6653
Epoch 133/5000
966/966 [==============================] - 0s - loss: 16.9751 - acc: 0.5673 - val_loss: 16.3147 - val_acc: 0.6653
Epoch 134/5000
966/966 [==============================] - 0s - loss: 16.9740 - acc: 0.5787 - val_loss: 16.3116 -

966/966 [==============================] - 0s - loss: 16.8587 - acc: 0.6304 - val_loss: 16.1724 - val_acc: 0.6653
Epoch 191/5000
966/966 [==============================] - 0s - loss: 16.8613 - acc: 0.6284 - val_loss: 16.1704 - val_acc: 0.6653
Epoch 192/5000
966/966 [==============================] - 0s - loss: 16.8623 - acc: 0.6325 - val_loss: 16.1684 - val_acc: 0.6653
Epoch 193/5000
966/966 [==============================] - 0s - loss: 16.8755 - acc: 0.6346 - val_loss: 16.1668 - val_acc: 0.6653
Epoch 194/5000
966/966 [==============================] - 0s - loss: 16.8596 - acc: 0.6449 - val_loss: 16.1655 - val_acc: 0.6653
Epoch 195/5000
966/966 [==============================] - 0s - loss: 16.8389 - acc: 0.6398 - val_loss: 16.1635 - val_acc: 0.6653
Epoch 196/5000
966/966 [==============================] - 0s - loss: 16.8569 - acc: 0.6356 - val_loss: 16.1615 - val_acc: 0.6653
Epoch 197/5000
966/966 [==============================] - 0s - loss: 16.8535 - acc: 0.6356 - val_loss: 16.1598 -

966/966 [==============================] - 0s - loss: 16.7961 - acc: 0.6625 - val_loss: 16.0766 - val_acc: 0.6653
Epoch 254/5000
966/966 [==============================] - 0s - loss: 16.7895 - acc: 0.6698 - val_loss: 16.0753 - val_acc: 0.6653
Epoch 255/5000
966/966 [==============================] - 0s - loss: 16.7698 - acc: 0.6615 - val_loss: 16.0740 - val_acc: 0.6653
Epoch 256/5000
966/966 [==============================] - 0s - loss: 16.7808 - acc: 0.6687 - val_loss: 16.0726 - val_acc: 0.6653
Epoch 257/5000
966/966 [==============================] - 0s - loss: 16.7820 - acc: 0.6636 - val_loss: 16.0715 - val_acc: 0.6653
Epoch 258/5000
966/966 [==============================] - 0s - loss: 16.7826 - acc: 0.6615 - val_loss: 16.0705 - val_acc: 0.6653
Epoch 259/5000
966/966 [==============================] - 0s - loss: 16.7806 - acc: 0.6615 - val_loss: 16.0694 - val_acc: 0.6653
Epoch 260/5000
966/966 [==============================] - 0s - loss: 16.7901 - acc: 0.6636 - val_loss: 16.0679 -

...

966/966 [==============================] - 0s - loss: 16.5641 - acc: 0.6698 - val_loss: 15.8266 - val_acc: 0.6777
Epoch 4721/5000
966/966 [==============================] - 0s - loss: 16.5465 - acc: 0.6812 - val_loss: 15.8266 - val_acc: 0.6777
Epoch 4722/5000
966/966 [==============================] - 0s - loss: 16.5701 - acc: 0.6698 - val_loss: 15.8266 - val_acc: 0.6777
Epoch 4723/5000
966/966 [==============================] - 0s - loss: 16.5518 - acc: 0.6749 - val_loss: 15.8266 - val_acc: 0.6777
Epoch 4724/5000
966/966 [==============================] - 0s - loss: 16.5780 - acc: 0.6760 - val_loss: 15.8266 - val_acc: 0.6777
Epoch 4725/5000
966/966 [==============================] - 0s - loss: 16.5661 - acc: 0.6698 - val_loss: 15.8266 - val_acc: 0.6777
Epoch 4726/5000
966/966 [==============================] - 0s - loss: 16.5632 - acc: 0.6677 - val_loss: 15.8266 - val_acc: 0.6777
Epoch 4727/5000
966/966 [==============================] - 0s - loss: 16.5696 - acc: 0.6739 - val_loss: 15

966/966 [==============================] - 0s - loss: 16.5671 - acc: 0.6625 - val_loss: 15.8261 - val_acc: 0.6777
Epoch 4784/5000
966/966 [==============================] - 0s - loss: 16.5853 - acc: 0.6708 - val_loss: 15.8261 - val_acc: 0.6777
Epoch 4785/5000
966/966 [==============================] - 0s - loss: 16.5640 - acc: 0.6729 - val_loss: 15.8261 - val_acc: 0.6777
Epoch 4786/5000
966/966 [==============================] - 0s - loss: 16.5644 - acc: 0.6718 - val_loss: 15.8261 - val_acc: 0.6777
Epoch 4787/5000
966/966 [==============================] - 0s - loss: 16.5696 - acc: 0.6625 - val_loss: 15.8261 - val_acc: 0.6777
Epoch 4788/5000
966/966 [==============================] - 0s - loss: 16.5570 - acc: 0.6739 - val_loss: 15.8261 - val_acc: 0.6777
Epoch 4789/5000
966/966 [==============================] - 0s - loss: 16.5589 - acc: 0.6718 - val_loss: 15.8261 - val_acc: 0.6777
Epoch 4790/5000
966/966 [==============================] - 0s - loss: 16.5551 - acc: 0.6708 - val_loss: 15

966/966 [==============================] - 0s - loss: 16.5761 - acc: 0.6718 - val_loss: 15.8260 - val_acc: 0.6777
Epoch 4846/5000
966/966 [==============================] - 0s - loss: 16.5587 - acc: 0.6605 - val_loss: 15.8260 - val_acc: 0.6777
Epoch 4847/5000
966/966 [==============================] - 0s - loss: 16.5592 - acc: 0.6656 - val_loss: 15.8260 - val_acc: 0.6777
Epoch 4848/5000
966/966 [==============================] - 0s - loss: 16.5657 - acc: 0.6708 - val_loss: 15.8260 - val_acc: 0.6777
Epoch 4849/5000
966/966 [==============================] - 0s - loss: 16.5629 - acc: 0.6749 - val_loss: 15.8260 - val_acc: 0.6777
Epoch 4850/5000
966/966 [==============================] - 0s - loss: 16.5722 - acc: 0.6667 - val_loss: 15.8260 - val_acc: 0.6777
Epoch 4851/5000
966/966 [==============================] - 0s - loss: 16.5749 - acc: 0.6760 - val_loss: 15.8260 - val_acc: 0.6777
Epoch 4852/5000
966/966 [==============================] - 0s - loss: 16.5628 - acc: 0.6625 - val_loss: 15

966/966 [==============================] - 0s - loss: 16.5689 - acc: 0.6749 - val_loss: 15.8258 - val_acc: 0.6777
Epoch 4909/5000
966/966 [==============================] - 0s - loss: 16.5886 - acc: 0.6667 - val_loss: 15.8257 - val_acc: 0.6777
Epoch 4910/5000
966/966 [==============================] - 0s - loss: 16.5710 - acc: 0.6625 - val_loss: 15.8257 - val_acc: 0.6777
Epoch 4911/5000
966/966 [==============================] - 0s - loss: 16.5530 - acc: 0.6749 - val_loss: 15.8257 - val_acc: 0.6777
Epoch 4912/5000
966/966 [==============================] - 0s - loss: 16.5686 - acc: 0.6708 - val_loss: 15.8257 - val_acc: 0.6777
Epoch 4913/5000
966/966 [==============================] - 0s - loss: 16.5783 - acc: 0.6739 - val_loss: 15.8257 - val_acc: 0.6777
Epoch 4914/5000
966/966 [==============================] - 0s - loss: 16.5444 - acc: 0.6698 - val_loss: 15.8257 - val_acc: 0.6777
Epoch 4915/5000
966/966 [==============================] - 0s - loss: 16.5582 - acc: 0.6801 - val_loss: 15

966/966 [==============================] - 0s - loss: 16.5661 - acc: 0.6770 - val_loss: 15.8252 - val_acc: 0.6777
Epoch 4972/5000
966/966 [==============================] - 0s - loss: 16.5677 - acc: 0.6770 - val_loss: 15.8253 - val_acc: 0.6777
Epoch 4973/5000
966/966 [==============================] - 0s - loss: 16.5496 - acc: 0.6729 - val_loss: 15.8252 - val_acc: 0.6777
Epoch 4974/5000
966/966 [==============================] - 0s - loss: 16.5641 - acc: 0.6749 - val_loss: 15.8252 - val_acc: 0.6777
Epoch 4975/5000
966/966 [==============================] - 0s - loss: 16.5744 - acc: 0.6791 - val_loss: 15.8252 - val_acc: 0.6777
Epoch 4976/5000
966/966 [==============================] - 0s - loss: 16.5409 - acc: 0.6739 - val_loss: 15.8252 - val_acc: 0.6777
Epoch 4977/5000
966/966 [==============================] - 0s - loss: 16.5481 - acc: 0.6739 - val_loss: 15.8252 - val_acc: 0.6777
Epoch 4978/5000
966/966 [==============================] - 0s - loss: 16.5656 - acc: 0.6708 - val_loss: 15